# The Second Agent - estimate the actual value of a product

## RAG (Retrieval Augmented Generation) based on a dataset of 400,000 scraped Amazon products

#### For our 2nd agent, we will be asking DeekSeek to estimate the price of one of our deals - and we will give it a hand.

It turns out that LLMs are really good at this! Out of the box, GPT-4o is off by an average of \$76.

But we can do even better: we'll provide it with some context, in the form of 5 similar products from our training dataset

Again I'll be going quite quickly through this - the idea is for you to run this yourself.

In [1]:
# imports

import os
import re
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm import tqdm

In [2]:
# environment

load_dotenv(override=True)
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace
# If you don't have a HuggingFace account, you can set one up for free at www.huggingface.co
# And then add the HF_TOKEN to your .env file as explained in the project README

hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# For following along at home:

Please download the files train.pkl and test.pkl from this Google Drive folder:  
https://drive.google.com/drive/folders/1t0YnoCXCbo2g08uWIOR6TPKR2-6Egb_g?usp=sharing

And place them in the parent directory (the directory called agentic).

In [4]:
# Load the training data

with open('../train.pkl', 'rb') as file:
    train = pickle.load(file)


In [5]:
print(f"There are {len(train):,} training items scraped from Amazon, and the first one is {train[0]}")

There are 400,000 training items scraped from Amazon, and the first one is <Delphi FG0166 Fuel Pump Module = $226.95>


# Now create a Chroma Datastore

Now we will use the free, open-source Vector database Chroma.  
We will create a Chroma datastore with 400,000 products from our training dataset.

In [6]:
client = chromadb.PersistentClient(path=DB)

# Introducing the SentenceTransformer Encoding LLM

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to 384 dimensional vectors and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

As an alternative, OpenAI provides a closed-source Embeddings model. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day",
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

torch.Size([4, 4])


In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["A room full of AI engineers"])[0]
print(vector.shape)
vector

(384,)


array([-9.29236319e-03, -2.76716854e-02,  2.08850093e-02,  5.88356331e-03,
        1.43282311e-02, -7.71259218e-02,  3.57389748e-02, -1.07448017e-02,
       -9.22502764e-03, -1.56571586e-02, -4.24381420e-02, -7.12145343e-02,
        5.86078018e-02, -1.77758262e-02, -5.20936064e-02,  6.14698976e-02,
       -7.55092409e-03, -8.00909176e-02, -3.97532471e-02, -9.76977423e-02,
       -1.56468358e-02,  1.16357766e-02,  1.56349875e-02, -5.24811298e-02,
        1.22509815e-03,  9.64261144e-02,  5.03962710e-02, -4.24284264e-02,
        6.73197359e-02, -5.70055582e-02,  2.85714846e-02,  3.03971935e-02,
        4.60026897e-02,  2.28838939e-02,  5.46884537e-02,  8.03021267e-02,
       -4.24176343e-02, -3.81376483e-02,  7.77059495e-02,  1.78595707e-02,
       -4.56924327e-02, -1.82922389e-02,  5.41159734e-02, -2.43166331e-02,
        5.32313697e-02,  7.15427995e-02, -8.75171460e-03, -1.05135977e-01,
        8.65057781e-02, -1.61130447e-02, -9.09638330e-02, -4.96833399e-02,
        7.29331980e-03, -

## With that background, let's populate our Chroma database

### By calculating vectors for 400,000 scraped products

In [10]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.text for item in train[i: i+1000]]
        vectors = model.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=vectors,
            metadatas=metadatas
        )
collection = client.get_or_create_collection(collection_name)

# Let's visualize the vectorized data

In [11]:
# It is very fun turning this up to 400_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 5_000 is safe!

MAXIMUM_DATAPOINTS = 5_000

In [12]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [13]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [ ]:
# Let's try a 2D chart
# TSNE stands for t-distributed Stochastic Neighbor Embedding - it's a common technique for reducing dimensionality of data

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [ ]:
# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [ ]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# Load in the test pickle file

with open('../test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
openrouter_url = os.environ['OPENROUTER_API_BASE']
openrouter_api_key = os.environ['OPENROUTER_API_KEY']

def preprocess(item):
    message = f"Reply with a 2-3 sentence summary of this product. This will be used to find similar products so it should be clear, concise, complete. Details:\n{item}"
    messages = [{"role": "user", "content": message}]
    response = completion(api_key=openrouter_api_key, base_url=openrouter_url, model="openai/gpt-oss-safeguard-20b", messages=messages)
    return response.choices[0].message.content

In [ ]:
print(test[1].text)
print(openrouter_url)

Motorcraft YB3125 Fan Clutch
Motorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Rank Automotive Automotive Replacement Engine Fan Clutches 583, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of the U.S. Learn More, Available October 10, 2007
https://openrouter.ai/api/v1


In [ ]:
preprocess(test[1].text)

'The Motorcraft YB3125 is a replacement engine fan clutch designed for use in a variety of automotive models. It measures 10\u202f×\u202f7.63\u202f×\u202f6.25\u202finches (25.1\u202f×\u202f20.1\u202f×\u202f15.5\u202fcm) and weighs approximately 5\u202flb, with an exterior painted finish. Manufactured by Motorcraft in China, this single‑unit package is available for domestic and selected international shipping.'

In [ ]:
# We need to give some context to GPT-5-mini by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [ ]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
def vector(item):
    text = preprocess(item.text)
    return model.encode(text)

In [ ]:
def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [ ]:
documents, prices = find_similars(test[1])
print(make_context(documents, prices))

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Motorcraft YB3076 Fan Clutch
Motorcraft Fan Clutches are designed and tested to meet OE specifications for durability and reliability under extreme conditions. These clutches feature fluid which enhances the fan to turn faster and helps the engine reach operating temperature. They are built to maximize performance and are made from high-quality materials. Thermostatically controlled viscous fan clutch Special fluid enhances the fan to turn faster Made of high quality Durable and reliable under extreme conditions Help the engine reach operating temperature Manufacturer Motorcraft, Brand Motorcraft, Weight 5 pounds, Dimensions 18 x 10.5 x 6 inches, Country of Origin China, model number Manufacturer Part Wattage 3.6 watts, Rank Automotive Automotive Replacement Engine Fan Clutches 138, Domestic Shipping can be shipped within U.S
Price is $249.78

Potentially related

In [ ]:
# Utility function that extracts a price from a response from GPT-4o-mini

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("blah blah the price is $99.99 blah")

99.99

In [ ]:
# The function for gpt-4-mini

def gpt_4_1_mini_rag(item):
    documents, prices = find_similars(item)
    response = completion(
        api_key=openrouter_api_key,
        base_url=openrouter_url,
        model="openrouter/openai/gpt-4.1-mini",
        messages=messages_for(item, documents, prices),
        max_tokens=16,
    )
    reply = response.choices[0].message.content
    return get_price(reply)

# function for gemini-2.5-pro
def gemini_2_5_pro_rag(item):
    documents, prices = find_similars(item)
    response = completion(
        api_key=openrouter_api_key,
        base_url=openrouter_url,
        model="openrouter/google/gemini-2.5-pro",
        messages=messages_for(item, documents, prices),
        max_tokens=16,
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
# How much does the Fan Clutch in test[1] actually cost, on Amazon?

print(test[1].price)
print(openrouter_url)
print(openrouter_api_key[:5])

225.11
https://openrouter.ai/api/v1
sk-or


In [ ]:
# Now let's call GPT-4o-mini using RAG, passing in 5 similar items from our Chroma datastore

gpt_4_1_mini_rag(test[1])

249.78

In [ ]:
gemini_2_5_pro_rag(test[1])

223.75

In [ ]:
# Try Gemini 2.5 Flash

def gemini_rag(item):
    documents, prices = find_similars(item)
    response = completion(
        api_key=openrouter_api_key,
        base_url=openrouter_url,
        model="openrouter/google/gemini-2.5-flash", 
        messages=messages_for(item, documents, prices), 
        max_tokens=8)
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
gemini_rag(test[1])

249.78

In [ ]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
from price_agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price(
    api_key=openrouter_api_key,
    base_url=openrouter_url,
    description="Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio"
)

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready


TypeError: FrontierAgent.price() got an unexpected keyword argument 'api_key'

In [ ]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")